# Crossvalidation experiments on MotionSense Resampled to 30Hz

This notebook will perform crossvalidation experiments using the MotionSense dataset at 30 Hz as training dataset. It will contain the following steps:

1. Quick load train, test and validation CSV subsets from the balanced MotionSense dataset at 30 Hz using `PandasDatasetsIO` helper
2. Quick load train, test and validation CSV subsets from other relevant datasets using `PandasDatasetsIO` helper
3. Subclassing the `Dataset` interface using `PandasMultiModalDataset`
4. Apply the fourier transform on all the datasets
5. Train SVM, KNN and Random Forest classification models on the MotionSense dataset in both time and frequency domains
6. Evaluate SVM, KNN and Random Forest classification models on other dataset in both time and frequency domains

The experiments will evaluate the performance of SVM, KNN and RF models trained on a balanced MotionSense dataset and tested on other datasets in both time and frequency domains.

## Common imports and definitions

In [1]:
from pathlib import Path  # For defining dataset Paths
import sys                # For include librep package

# This must be done if librep is not installed via pip,
# as this directory (examples) is appart from librep package root
sys.path.append("..")

# Third party imports
import pandas as pd
import numpy as np

# Librep imports
from librep.utils.dataset import PandasDatasetsIO          # For quick load train, test and validation CSVs
from librep.datasets.multimodal import PandasMultiModalDataset # Wrap CSVs to librep's `Dataset` interface

2022-09-13 13:04:52.376366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-13 13:04:52.376453: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data from base dataset
Change the path here to use in other dataset as base

In [2]:
# Path for KuHar resampled to 30Hz view with the same activities (and labels numbers)
# It is assumed that the directory will contain (train.csv, test.csv and validation.csv)
dataset_path = Path("../data/views/MotionSense/resampled_view_30Hz")

Once paths is defined, we can load the CSV as pandas dataframes

In [3]:
# Kuhar dataframes
train_base, validation_base, test_base = PandasDatasetsIO(dataset_path).load()

Letś take a look in the train dataframes

In [4]:
train_base.head()

,Unnamed: 0,userAcceleration.x-0,userAcceleration.x-1,userAcceleration.x-2,userAcceleration.x-3,userAcceleration.x-4,userAcceleration.x-5,userAcceleration.x-6,userAcceleration.x-7,userAcceleration.x-8,...,rotationRate.z-85,rotationRate.z-86,rotationRate.z-87,rotationRate.z-88,rotationRate.z-89,activity code,length,trial_code,index,user
0,0,-0.186833,-0.179195,-0.226435,-0.234763,-0.267824,-0.234534,-0.235421,-0.133759,-0.297125,...,0.153844,0.456858,0.898804,1.139253,0.275556,0,150,1,150,11
1,1,-0.054442,0.260099,0.022933,0.019339,0.148599,-0.036896,-0.125777,-0.110877,0.016260,...,0.848919,0.559802,0.253026,0.858864,0.799075,0,150,1,900,12
2,2,-0.007696,-0.009515,0.051284,-0.082342,0.046316,0.062557,-0.032338,-0.108787,-0.090480,...,0.789453,0.495270,-0.042529,-0.161110,0.129157,0,150,1,1050,21
3,3,-0.435023,-0.557701,-0.284523,0.142448,0.545683,0.363495,0.006622,-0.042397,-0.412486,...,-1.073667,-0.531939,-0.302297,0.086762,0.600790,0,150,2,150,17
4,4,0.098066,0.398057,0.321284,-0.024039,0.372623,0.302234,0.199685,0.285311,0.319281,...,0.884322,1.476072,1.630557,0.927239,0.672827,0,150,11,450,21


## Creating a Librep dataset from pandas dataframes

Change the features to use in other datasets

In [5]:
# MotionSense features to select
features = [
    "userAcceleration.x",
    "userAcceleration.y",
    "userAcceleration.z",
    "rotationRate.x",
    "rotationRate.y",
    "rotationRate.z"
]

# Creating the train dataset

combined_train_dset = PandasMultiModalDataset(
    pd.concat([train_base, validation_base, test_base], ignore_index=True),
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

## Inspect sample

In [6]:
# Lets print the first sample of combined_train dataset.
# Is a tuple, with an vector of 540 elements as first element and the label as second
x = combined_train_dset[0]
print(x)

(array([-1.86833389e-01, -1.79195359e-01, -2.26435131e-01, -2.34763436e-01,
       -2.67823557e-01, -2.34534448e-01, -2.35421098e-01, -1.33759301e-01,
       -2.97124572e-01, -3.19298325e-01,  8.95068153e-03,  4.13667107e-02,
       -4.45930376e-01, -2.28045572e-01,  5.05584862e-02,  1.88233135e-01,
       -3.05589990e-02, -7.44910767e-02,  3.59969568e-02, -5.25122747e-02,
       -1.20435540e-01, -2.99962061e-01, -4.50567186e-01,  1.46823014e-01,
        1.64267921e-02,  1.00866929e-01,  1.56137079e-01, -9.38140351e-02,
       -2.79031325e-01, -2.71325809e-01, -1.99438544e-01, -3.22355437e-01,
       -2.81747706e-01, -1.88338075e-01, -1.75930166e-01, -2.58367241e-01,
       -4.13807163e-01, -6.38969102e-01, -6.66223601e-02,  6.10918606e-01,
        1.23116570e-01,  7.50942481e-02,  1.32656873e-01,  1.36065249e-01,
       -7.60303034e-02, -1.99809749e-01,  8.69800753e-03,  3.54754458e-01,
       -2.36973734e-01, -3.67774663e-01, -2.45674023e-01,  1.50712010e-01,
        1.44587441e-01, 

In [7]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[1]}")

The sample 0: [-1.86833389e-01 -1.79195359e-01 -2.26435131e-01 -2.34763436e-01
 -2.67823557e-01 -2.34534448e-01 -2.35421098e-01 -1.33759301e-01
 -2.97124572e-01 -3.19298325e-01  8.95068153e-03  4.13667107e-02
 -4.45930376e-01 -2.28045572e-01  5.05584862e-02  1.88233135e-01
 -3.05589990e-02 -7.44910767e-02  3.59969568e-02 -5.25122747e-02
 -1.20435540e-01 -2.99962061e-01 -4.50567186e-01  1.46823014e-01
  1.64267921e-02  1.00866929e-01  1.56137079e-01 -9.38140351e-02
 -2.79031325e-01 -2.71325809e-01 -1.99438544e-01 -3.22355437e-01
 -2.81747706e-01 -1.88338075e-01 -1.75930166e-01 -2.58367241e-01
 -4.13807163e-01 -6.38969102e-01 -6.66223601e-02  6.10918606e-01
  1.23116570e-01  7.50942481e-02  1.32656873e-01  1.36065249e-01
 -7.60303034e-02 -1.99809749e-01  8.69800753e-03  3.54754458e-01
 -2.36973734e-01 -3.67774663e-01 -2.45674023e-01  1.50712010e-01
  1.44587441e-01 -1.08431514e-01 -1.82670965e-02 -8.24199145e-02
 -3.54580921e-02 -2.73086366e-01 -3.19687428e-01 -1.72529165e-01
  2.8114138

## Fourier Transform

In [8]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [9]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")

### Use FFT in the base dataset

In [10]:
combined_train_dset_fft = transformer(combined_train_dset)

In [11]:
combined_train_dset[:][0]

array([[-0.18683339, -0.17919536, -0.22643513, ...,  0.89880418,
         1.13925318,  0.27555584],
       [-0.05444212,  0.26009924,  0.0229331 , ...,  0.25302617,
         0.85886433,  0.79907488],
       [-0.00769597, -0.00951478,  0.05128415, ..., -0.04252899,
        -0.16110965,  0.12915699],
       ...,
       [-0.31903905, -0.01283569, -0.03048265, ..., -0.4407308 ,
        -0.45499816, -0.90586541],
       [ 0.34836533,  0.52526862,  0.05997598, ..., -0.29225946,
        -0.48266957,  0.63102237],
       [ 0.09118243,  0.2065321 ,  0.38206896, ...,  0.3682671 ,
         0.32344069,  0.98400094]])

In [12]:
combined_train_dset_fft[:][0]

array([[7.8524508 , 1.97250947, 0.93442469, ..., 1.89050909, 0.83709156,
        1.20154337],
       [0.6293334 , 2.21672832, 1.13328689, ..., 1.16967075, 3.53071988,
        1.06557213],
       [3.1927866 , 1.55762082, 0.60235897, ..., 0.55560622, 0.71680657,
        0.39816055],
       ...,
       [1.2537858 , 1.29129674, 0.93891406, ..., 1.1729674 , 0.78012212,
        1.21739774],
       [3.1272222 , 2.91728302, 2.09904721, ..., 1.26141987, 5.8859495 ,
        1.62984581],
       [1.5251454 , 1.44493422, 1.36297853, ..., 1.90046437, 1.3680962 ,
        1.49000302]])

## Loading data from a target dataset
In this case we use the MotionSense dataset resampled at 30 Hz

In [13]:
dataset_path = Path("../data/views/KuHar/resampled_view_30Hz")

In [14]:
# Kuhar dataframes
train_target, validation_target, test_target = PandasDatasetsIO(dataset_path).load()

In [15]:
train_base.head()

,Unnamed: 0,userAcceleration.x-0,userAcceleration.x-1,userAcceleration.x-2,userAcceleration.x-3,userAcceleration.x-4,userAcceleration.x-5,userAcceleration.x-6,userAcceleration.x-7,userAcceleration.x-8,...,rotationRate.z-85,rotationRate.z-86,rotationRate.z-87,rotationRate.z-88,rotationRate.z-89,activity code,length,trial_code,index,user
0,0,-0.186833,-0.179195,-0.226435,-0.234763,-0.267824,-0.234534,-0.235421,-0.133759,-0.297125,...,0.153844,0.456858,0.898804,1.139253,0.275556,0,150,1,150,11
1,1,-0.054442,0.260099,0.022933,0.019339,0.148599,-0.036896,-0.125777,-0.110877,0.016260,...,0.848919,0.559802,0.253026,0.858864,0.799075,0,150,1,900,12
2,2,-0.007696,-0.009515,0.051284,-0.082342,0.046316,0.062557,-0.032338,-0.108787,-0.090480,...,0.789453,0.495270,-0.042529,-0.161110,0.129157,0,150,1,1050,21
3,3,-0.435023,-0.557701,-0.284523,0.142448,0.545683,0.363495,0.006622,-0.042397,-0.412486,...,-1.073667,-0.531939,-0.302297,0.086762,0.600790,0,150,2,150,17
4,4,0.098066,0.398057,0.321284,-0.024039,0.372623,0.302234,0.199685,0.285311,0.319281,...,0.884322,1.476072,1.630557,0.927239,0.672827,0,150,11,450,21


In [16]:
# KuHar features to select
features = [
    "accel-x",
    "accel-y",
    "accel-z",
    "gyro-x",
    "gyro-y",
    "gyro-z"
]

combined_df = pd.concat([train_target, validation_target, test_target], ignore_index=True)
df2 = combined_df.query("`activity code` == 0 or `activity code` == 1 or `activity code` == 11 or `activity code` == 14  or `activity code` == 15 or `activity code` == 16")
df2.loc[ df2['activity code'] == 0, 'activity code'] = 3
df2.loc[ df2['activity code'] == 1, 'activity code'] = 2
df2.loc[ df2['activity code'] == 11, 'activity code'] = 4
df2.loc[ df2['activity code'] == 14, 'activity code'] = 5
df2.loc[ df2['activity code'] == 15, 'activity code'] = 1
df2.loc[ df2['activity code'] == 16, 'activity code'] = 0

# Creating the train dataset

combined_target_dset = PandasMultiModalDataset(
    df2,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

In [17]:
# Lets print the first sample of combined_target dataset.
# Is a tuple, with an vector of 180 elements as first element and the label as second
x = combined_target_dset[0]
print(x)

(array([ 3.62008887e-03, -2.36884079e-02, -2.65723909e-03,  1.03708716e-02,
       -5.85826359e-03,  1.04230261e-02, -1.95931668e-02,  3.02941467e-04,
        4.54334895e-02, -4.61346122e-03, -4.32958544e-02, -2.11811327e-03,
       -3.24331746e-02, -9.64189479e-04,  2.38423378e-02,  1.46571666e-02,
       -6.33010786e-02, -3.09536555e-02, -6.09642095e-03, -2.23327124e-02,
        4.73811398e-02,  8.10745709e-03, -1.46291624e-03, -3.52547565e-02,
       -3.65486640e-02,  3.89222511e-02, -5.64525903e-02,  1.71065679e-02,
       -1.88765233e-02,  1.56273664e-02,  6.68170176e-02,  1.86759574e-02,
       -4.15165877e-02,  1.55675209e-02,  4.57480490e-03, -1.56470432e-02,
        2.86984361e-02, -3.88362938e-03, -1.85153493e-02, -9.91992429e-03,
        1.05702633e-02, -5.34487311e-03,  2.19452625e-02,  2.31337120e-02,
       -7.65234924e-03, -5.43440699e-02, -2.68535392e-02,  1.13130045e-02,
        2.85217945e-02,  1.84220024e-02,  1.95164920e-02,  4.88774164e-03,
        6.26222956e-03, 

In [18]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[1]}")

The sample 0: [ 3.62008887e-03 -2.36884079e-02 -2.65723909e-03  1.03708716e-02
 -5.85826359e-03  1.04230261e-02 -1.95931668e-02  3.02941467e-04
  4.54334895e-02 -4.61346122e-03 -4.32958544e-02 -2.11811327e-03
 -3.24331746e-02 -9.64189479e-04  2.38423378e-02  1.46571666e-02
 -6.33010786e-02 -3.09536555e-02 -6.09642095e-03 -2.23327124e-02
  4.73811398e-02  8.10745709e-03 -1.46291624e-03 -3.52547565e-02
 -3.65486640e-02  3.89222511e-02 -5.64525903e-02  1.71065679e-02
 -1.88765233e-02  1.56273664e-02  6.68170176e-02  1.86759574e-02
 -4.15165877e-02  1.55675209e-02  4.57480490e-03 -1.56470432e-02
  2.86984361e-02 -3.88362938e-03 -1.85153493e-02 -9.91992429e-03
  1.05702633e-02 -5.34487311e-03  2.19452625e-02  2.31337120e-02
 -7.65234924e-03 -5.43440699e-02 -2.68535392e-02  1.13130045e-02
  2.85217945e-02  1.84220024e-02  1.95164920e-02  4.88774164e-03
  6.26222956e-03  1.74586067e-02  4.62809500e-03 -4.00069874e-02
  1.28236155e-03  1.63634494e-02 -7.79265175e-04  3.31124115e-02
 -6.2073271

In [19]:
combined_target_dset_fft = transformer(combined_target_dset)

In [20]:
combined_target_dset[:][0]

array([[ 3.62008887e-03, -2.36884079e-02, -2.65723909e-03, ...,
         2.38507494e-03,  4.01598763e-03,  3.37713756e-03],
       [-5.82278332e-03,  1.24938221e-02, -1.25032413e-02, ...,
        -3.11192249e-03,  2.45944768e-03,  3.05583499e-03],
       [-3.92784081e-02,  3.86433489e-03,  8.92735084e-03, ...,
         4.96789877e-03,  3.30600047e-03,  3.44192435e-03],
       ...,
       [-1.99585460e+00,  4.04999945e-01, -3.54061931e+00, ...,
        -2.74810522e-01, -7.06500809e-01,  3.30508022e-01],
       [ 7.92721170e+00,  1.51318127e+01,  5.00892725e+00, ...,
         4.63478777e-01, -1.88581177e-01,  2.97794095e-01],
       [-9.39144061e+00, -7.32196700e+00,  6.82293387e-01, ...,
         1.79411423e-01,  2.67349940e-02, -2.87605829e-01]])

In [21]:
combined_target_dset_fft[:][0]

array([[3.13384994e-02, 1.68121633e-01, 9.05549650e-02, ...,
        5.92594004e-03, 1.10474894e-02, 7.03162882e-03],
       [2.30704315e-02, 1.23651598e-01, 6.28149230e-02, ...,
        8.26119084e-03, 3.20484735e-03, 5.24385686e-03],
       [7.81909007e-02, 7.24224293e-02, 1.34035957e-01, ...,
        1.17237633e-02, 8.03545237e-03, 2.37891069e-02],
       ...,
       [2.49188042e+01, 1.04846176e+01, 2.49600827e+01, ...,
        3.11570416e-01, 9.85833285e-01, 5.65979433e-01],
       [4.25616557e+01, 3.97250479e+01, 3.13979932e+01, ...,
        1.17911618e+00, 1.55453648e+00, 2.65968394e+00],
       [3.18897913e+01, 2.62539740e+01, 2.49088799e+01, ...,
        1.44891605e+00, 3.51355869e+00, 1.50627152e+00]])

## Train and evaluate Random Forest classifier

In [22]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, estimator_creation_kwags ={'n_estimators':100} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [23]:
result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663074320.5761254
    result:
    -   accuracy: 0.3356918238993711
        f1 score (macro): 0.24991713060357815
        f1 score (micro): 0.3356918238993711
        f1 score (weighted): 0.42146651719516404
    run id: 1
    start: 1663074313.7588165
    time taken: 6.8173089027404785
-   end: 1663074327.257351
    result:
    -   accuracy: 0.3443396226415094
        f1 score (macro): 0.26458349592636676
        f1 score (micro): 0.3443396226415094
        f1 score (weighted): 0.4240957493566522
    run id: 2
    start: 1663074320.5761275
    time taken: 6.681223392486572
-   end: 1663074333.8880286
    result:
    -   accuracy: 0.3356918238993711
        f1 score (macro): 0.2528108622873118
        f1 score (micro): 0.3356918238993711
        f1 score (weighted): 0.4185727855114304
    run id: 3
    start: 1663074327.2573526
    time taken: 6.630676031112671



In [24]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663074338.1604311
    result:
    -   accuracy: 0.34276729559748426
        f1 score (macro): 0.25787849864047124
        f1 score (micro): 0.34276729559748426
        f1 score (weighted): 0.4276560925544972
    run id: 1
    start: 1663074333.89288
    time taken: 4.2675511837005615
-   end: 1663074342.441355
    result:
    -   accuracy: 0.35691823899371067
        f1 score (macro): 0.2848265493963091
        f1 score (micro): 0.35691823899371067
        f1 score (weighted): 0.42900992859111237
    run id: 2
    start: 1663074338.1604328
    time taken: 4.280922174453735
-   end: 1663074346.737685
    result:
    -   accuracy: 0.3466981132075472
        f1 score (macro): 0.26827126586712957
        f1 score (micro): 0.3466981132075472
        f1 score (weighted): 0.4251249605479648
    run id: 3
    start: 1663074342.441357
    time taken: 4.296328067779541



## Train and evaluate Support Vector Machine classifier

In [25]:
#from librep.estimators import SVC
from sklearn.svm import SVC

experiment = SimpleTrainEvalWorkflow(estimator=SVC, estimator_creation_kwags ={'C':3.0, 'kernel':"rbf"} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663074352.0829716
    result:
    -   accuracy: 0.3309748427672956
        f1 score (macro): 0.18041563922780726
        f1 score (micro): 0.3309748427672956
        f1 score (weighted): 0.48153404630678387
    run id: 1
    start: 1663074346.7436316
    time taken: 5.339339971542358
-   end: 1663074357.498837
    result:
    -   accuracy: 0.3309748427672956
        f1 score (macro): 0.18041563922780726
        f1 score (micro): 0.3309748427672956
        f1 score (weighted): 0.48153404630678387
    run id: 2
    start: 1663074352.0829742
    time taken: 5.415862798690796
-   end: 1663074362.6223373
    result:
    -   accuracy: 0.3309748427672956
        f1 score (macro): 0.18041563922780726
        f1 score (micro): 0.3309748427672956
        f1 score (weighted): 0.48153404630678387
    run id: 3
    start: 1663074357.4988394
    time taken: 5.12349796295166



In [26]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663074363.6563182
    result:
    -   accuracy: 0.3309748427672956
        f1 score (macro): 0.184584311082615
        f1 score (micro): 0.3309748427672956
        f1 score (weighted): 0.47736537445197624
    run id: 1
    start: 1663074362.6278028
    time taken: 1.028515338897705
-   end: 1663074364.6899588
    result:
    -   accuracy: 0.3309748427672956
        f1 score (macro): 0.184584311082615
        f1 score (micro): 0.3309748427672956
        f1 score (weighted): 0.47736537445197624
    run id: 2
    start: 1663074363.65632
    time taken: 1.0336387157440186
-   end: 1663074365.7185357
    result:
    -   accuracy: 0.3309748427672956
        f1 score (macro): 0.184584311082615
        f1 score (micro): 0.3309748427672956
        f1 score (weighted): 0.47736537445197624
    run id: 3
    start: 1663074364.6899605
    time taken: 1.0285751819610596



## Train and evaluate K Neighbors Classifier classifier

In [27]:
#from librep.estimators import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

experiment = SimpleTrainEvalWorkflow(estimator=KNeighborsClassifier, estimator_creation_kwags ={'n_neighbors' :1} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, combined_target_dset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663074366.1943138
    result:
    -   accuracy: 0.23349056603773585
        f1 score (macro): 0.21576655133093828
        f1 score (micro): 0.23349056603773585
        f1 score (weighted): 0.2512145807445334
    run id: 1
    start: 1663074365.7244139
    time taken: 0.46989989280700684
-   end: 1663074366.2803133
    result:
    -   accuracy: 0.23349056603773585
        f1 score (macro): 0.21576655133093828
        f1 score (micro): 0.23349056603773585
        f1 score (weighted): 0.2512145807445334
    run id: 2
    start: 1663074366.1943178
    time taken: 0.08599543571472168
-   end: 1663074366.358634
    result:
    -   accuracy: 0.23349056603773585
        f1 score (macro): 0.21576655133093828
        f1 score (micro): 0.23349056603773585
        f1 score (weighted): 0.2512145807445334
    run id: 3
    start: 1663074366.2803152
    time taken: 0.07831883430480957



In [28]:
result = multi_run_experiment(combined_train_dset_fft, combined_target_dset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1663074366.410654
    result:
    -   accuracy: 0.45990566037735847
        f1 score (macro): 0.4255125540235018
        f1 score (micro): 0.45990566037735847
        f1 score (weighted): 0.49429876673121526
    run id: 1
    start: 1663074366.3646984
    time taken: 0.045955657958984375
-   end: 1663074366.469787
    result:
    -   accuracy: 0.45990566037735847
        f1 score (macro): 0.4255125540235018
        f1 score (micro): 0.45990566037735847
        f1 score (weighted): 0.49429876673121526
    run id: 2
    start: 1663074366.4106557
    time taken: 0.05913114547729492
-   end: 1663074366.5152614
    result:
    -   accuracy: 0.45990566037735847
        f1 score (macro): 0.4255125540235018
        f1 score (micro): 0.45990566037735847
        f1 score (weighted): 0.49429876673121526
    run id: 3
    start: 1663074366.469789
    time taken: 0.04547238349914551

